In [58]:
import boto3
import pandas as pd
import configparser
import os
import json

In [78]:
# setting configuration filepath
filepath = os.path.join('..', 'dwh.cfg')

# loading configuration file
config = configparser.ConfigParser()
config.read_file(open(filepath))

In [79]:
KEY = config.get('AWS', 'KEY')
SECRET = config.get('AWS', 'SECRET')

In [13]:
session = boto3.session.Session(
    aws_access_key_id=KEY,
    aws_secret_access_key=SECRET,
    region_name='us-west-2'
)

In [14]:
iam_client = session.client('iam')

In [24]:
resp = iam_client.list_policies(MaxItems=1000)
df_policies = pd.DataFrame(resp['Policies'])

In [54]:
df_s3_policy = df_policies.query("PolicyName == 'AmazonS3ReadOnlyAccess'").reset_index()
df_s3_policy

,index,PolicyName,PolicyId,Arn,Path,DefaultVersionId,AttachmentCount,PermissionsBoundaryUsageCount,IsAttachable,CreateDate,UpdateDate
0,233,AmazonS3ReadOnlyAccess,ANPAIZTJ4DXE7G6AGAE6M,arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess,/,v2,0,0,True,2015-02-06 18:40:59+00:00,2021-09-27 20:24:58+00:00


In [94]:
%%writefile ../aws/sparkifydwh_role_create.py

def sparkifydwh_role_create (session):
    iam_client = session.client('iam')

    role_name = 'sparkifydwh_role'
    s3_read_only_arn = 'arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess'
    sparkifydwh_role = None

    try:
        sparkifydwh_role = iam_client.get_role(RoleName=role_name)
        print('Role already exists.')
    except iam_client.exceptions.NoSuchEntityException as error:
        sparkifydwh_role = iam_client.create_role(
            RoleName=role_name,
            Description='Define the aws accesses for the sparkify data warehouse',
            AssumeRolePolicyDocument=json.dumps({
                "Version": "2012-10-17",
                "Statement": {
                    "Effect": "Allow",
                    "Action": "sts:AssumeRole",
                    "Principal": {
                        "Service": "redshift.amazonaws.com"
                    }
                }
            })
        )

        print('Role created.')
    except Exception as error:
        print(error)
    
    iam_client.attach_role_policy(RoleName=role_name, PolicyArn=s3_read_only_arn)
    
    return sparkifydwh_role['Role']['Arn'], role_name 

Writing ../aws/sparkifydwh_role_create.py


In [100]:
%%writefile ../aws/sparkifydwh_role_delete.py

def sparkifydwh_role_delete (session):
    iam_client = session.client('iam')

    role_name = 'sparkifydwh_role'
    s3_read_only_arn = 'arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess'

    iam_client.detach_role_policy(RoleName=role_name, PolicyArn=s3_read_only_arn)
    iam_client.delete_role(RoleName=role_name)

Writing ../aws/sparkifydwh_role_delete.py


In [98]:
sparkifydwh_role_delete(session)

In [99]:
arn, role_name = create_sparkifydwh_role(session)
(arn, role_name)

Role created.


('arn:aws:iam::817838951406:role/sparkifydwh_role', 'sparkifydwh_role')

In [77]:
config['IAM_ROLE']['ARN'] = sparkifydwh_rol['Role']['Arn']
config.write(open(filepath, 'w'))